In [307]:
import os
import pandas as pd
import urllib.parse
import qrcode
import re
print ("Biblioteca importada com sucesso!")

Biblioteca importada com sucesso!


In [325]:
#Definir constantes para caminhos e URLs

URL_BASE_QR = "https://datacanuto.github.io/galeria-digital-artes"

# Obter diretório base do repositório (onde está o notebook)
DIRETORIO_BASE = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()

# Definir caminhos relativos ao repositório
IMG_PATH = os.path.join(DIRETORIO_BASE, "assets", "img")
DIRETORIO_SAIDA = os.path.join(DIRETORIO_BASE, "links_obras")
QR_EXPORT_DIR = os.path.join(DIRETORIO_BASE, "qr_codes_export")
DADOS = os.path.join(DIRETORIO_BASE, "dados_obras.csv")
DADOS_TRATADOS = os.path.join(DIRETORIO_BASE, "obras_com_links.csv")
HTML = os.path.join(DIRETORIO_BASE, "index.html")

# Criar diretório para exportação de QR codes
os.makedirs(QR_EXPORT_DIR, exist_ok=True)
print(f"Diretório base: {DIRETORIO_BASE}")
print(f"Diretório de imagens: {IMG_PATH}")
print(f"Diretório de exportação de QR codes: {QR_EXPORT_DIR}")

Diretório base: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes
Diretório de imagens: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\assets\img
Diretório de exportação de QR codes: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\qr_codes_export


In [326]:
df = pd.read_csv(DADOS)
df = pd.DataFrame(df)
print("Dados carregados com sucesso!")
print("Total de registros:", len(df))
df.head()

Dados carregados com sucesso!
Total de registros: 62


,ITEM,TELAS,TÉCNICA,DIMENSÃO,ANO,VALOR (R$),VALOR_PIX,PARCELAMENTO,CATEGORIA
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,"6.000,00","5.700,00",5% À VISTA/ 6 VEZES,TELAS
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2024,"8.400,00","7.980,00",5% À VISTA/ 6 VEZES,TELAS
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2024,"3.860,00","3.667,00",5% À VISTA/ 6 VEZES,TELAS
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,"3.500,00","3.325,00",5% À VISTA/ 6 VEZES,TELAS
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2024,"7.000,00","6.650,00",5% À VISTA/ 6 VEZES,TELAS


In [327]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ITEM          62 non-null     int64 
 1   TELAS         62 non-null     object
 2   TÉCNICA       62 non-null     object
 3   DIMENSÃO      62 non-null     object
 4   ANO           62 non-null     int64 
 5   VALOR (R$)    62 non-null     object
 6   VALOR_PIX     62 non-null     object
 7   PARCELAMENTO  62 non-null     object
 8   CATEGORIA     62 non-null     object
dtypes: int64(2), object(7)
memory usage: 4.5+ KB


In [328]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.lower()

print(df.columns)

Index(['item', 'telas', 'técnica', 'dimensão', 'ano', 'valor (r$)',
       'valor_pix', 'parcelamento', 'categoria'],
      dtype='object')


In [329]:
img_list = os.listdir(IMG_PATH)
img_fullpath = [os.path.join(IMG_PATH, img) for img in img_list]

# Criar dicionário para mapear índice com imagem
img_dict = {i+1: img for i, img in enumerate(img_list)}
df['fotos'] = df['item'].map(img_dict)

In [330]:
# Limpar e converter colunas de valor para float
# Substituir 'acervo pessoal', '-' e valores vazios por 0

def limpar_valor_brasileiro(valor):
    """Converte valores no formato brasileiro (ex: '6.000,00') para float"""
    if pd.isna(valor):
        return 0.0
    
    valor_str = str(valor).strip().lower()
    
    # Verificar se é 'acervo pessoal', vazio ou '-'
    if valor_str in ['acervo pessoal', '', 'nan', '-', '  -   ']:
        return 0.0
    
    # Remover espaços
    valor_str = valor_str.replace(' ', '')
    
    # Formato brasileiro: "6.000,00" 
    # Remover pontos de milhar e trocar vírgula por ponto decimal
    if ',' in valor_str:
        # Se tem vírgula, é o separador decimal
        valor_str = valor_str.replace('.', '')  # Remove pontos (milhares)
        valor_str = valor_str.replace(',', '.')  # Vírgula vira ponto decimal
    # Se não tem vírgula mas tem ponto, assumir que já está correto
    
    try:
        valor_float = float(valor_str)
        # Verificar se o valor está muito alto (multiplicado por 100)
        # Valores normais de arte: 350 a 10.000 reais
        if valor_float > 100000:  # Se maior que 100k, dividir por 100
            valor_float = valor_float / 100
        return round(valor_float, 2)
    except:
        return 0.0

# Processar coluna 'valor (r$)'
df['valor (r$)'] = df['valor (r$)'].apply(limpar_valor_brasileiro)

# Processar coluna 'valor_pix' (se existir)
if 'valor_pix' in df.columns:
    df['valor_pix'] = df['valor_pix'].apply(limpar_valor_brasileiro)

print("Colunas de valor convertidas para float com 2 casas decimais!")
print("\nExemplo dos dados:")

if 'valor_pix' in df.columns:
    print(df[['telas', 'valor (r$)', 'valor_pix']].head(15))
else:
    print(df[['telas', 'valor (r$)']].head(15))

Colunas de valor convertidas para float com 2 casas decimais!

Exemplo dos dados:
                  telas  valor (r$)  valor_pix
0    PORTAS E JANELAS 1      6000.0     5700.0
1    PORTAS E JANELAS 2      8400.0     7980.0
2    PORTAS E JANELAS 3      3860.0     3667.0
3    PORTAS E JANELAS 4      3500.0     3325.0
4   PORTAS E JANELAS 5       7000.0     6650.0
5       CASA CURITIBA 1      6720.0     6384.0
6       CASA CURITIBA 2      6720.0     6384.0
7        FUSCA VERMELHO      3500.0     3325.0
8               WALKING      7500.0     7125.0
9                SUCATA         0.0        0.0
10         STREET DANCE         0.0        0.0
11              PILATES      3500.0     3325.0
12            OLD TIMES      3500.0     3325.0
13        CENAS URBANAS      4800.0     4560.0
14          FERRO VELHO      6720.0     6384.0


In [331]:
import numpy as np
df['status'] = np.where(df['valor (r$)'] != 0, 'disponível', 'acervo pessoal')

In [332]:
import re
df['op_parcelamento'] = df['parcelamento'].apply(lambda x: re.search(r'/\s*(\d+)', str(x)).group(1) if pd.notna(x) and re.search(r'/\s*(\d+)', str(x)) else '0')

In [339]:
# Carregar links do CSV gerado pela API (Crédito)
import os
csv_com_links = os.path.join(DIRETORIO_BASE, "obras_com_links_api.csv")

if os.path.exists(csv_com_links):
    df_links = pd.read_csv(csv_com_links)
    df_links.columns = df_links.columns.str.strip().str.lower()
    df['link_mp'] = df_links['link_mp']
    print(f"✅ {len([l for l in df['link_mp'] if pd.notna(l) and l != 'None'])} links de crédito carregados")
else:
    print(f"⚠️  Arquivo {csv_com_links} não encontrado!")
    print("Execute primeiro: .\\venv\\Scripts\\python.exe mercado_pago_api.py --mode prod")

# Carregar links PIX do CSV gerado pela API
csv_com_links_pix = os.path.join(DIRETORIO_BASE, "obras_com_links_pix_api.csv")

if os.path.exists(csv_com_links_pix):
    df_links_pix = pd.read_csv(csv_com_links_pix)
    df_links_pix.columns = df_links_pix.columns.str.strip().str.lower()
    df['link_mp_pix'] = df_links_pix['link_mp_pix']
    print(f"✅ {len([l for l in df['link_mp_pix'] if pd.notna(l) and l != 'None'])} links PIX carregados")
else:
    print(f"⚠️  Arquivo {csv_com_links_pix} não encontrado!")
    print("Execute primeiro: .\\venv\\Scripts\\python.exe mercado_pago_api_pix.py --mode prod")

print(f"\nExemplo link crédito: {df['link_mp'].iloc[0]}")
print(f"Exemplo link PIX: {df['link_mp_pix'].iloc[0]}")

✅ 60 links de crédito carregados
✅ 60 links PIX carregados

Exemplo link crédito: https://www.mercadopago.com.br/checkout/v1/redirect?pref_id=3041576384-582408e8-e3de-487f-bf63-59d884161692
Exemplo link PIX: https://www.mercadopago.com.br/checkout/v1/redirect?pref_id=3041576384-aa0cb8fe-48ed-4f4e-a9f0-60380622dd92


In [335]:
df.head(20)

,item,telas,técnica,dimensão,ano,valor (r$),valor_pix,parcelamento,categoria,fotos,status,op_parcelamento,link_mp
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,6000.0,5700.0,5% À VISTA/ 6 VEZES,TELAS,REF 01...PORTAS E JANELAS 1.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2024,8400.0,7980.0,5% À VISTA/ 6 VEZES,TELAS,REF 02...PORTAS E JANELAS 2.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2024,3860.0,3667.0,5% À VISTA/ 6 VEZES,TELAS,REF 03...PORTAS E JANELAS 3.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,3500.0,3325.0,5% À VISTA/ 6 VEZES,TELAS,REF 04 ...PORTAS E JANELAS 4.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2024,7000.0,6650.0,5% À VISTA/ 6 VEZES,TELAS,REF 05 - PORTAS E JANELAS 5.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
5,6,CASA CURITIBA 1,ACRÍLICA SOBRE TELA,70 X 120,2019,6720.0,6384.0,5% À VISTA/ 6 VEZES,TELAS,REF 06 - CASA CURITIBA 1.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
6,7,CASA CURITIBA 2,ACRÍLICA SOBRE TELA,70 X 120,2019,6720.0,6384.0,5% À VISTA/ 6 VEZES,TELAS,REF 07 - CASA CURITIBA 2.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
7,8,FUSCA VERMELHO,ACRÍLICA SOBRE TELA,70 X 50,2006,3500.0,3325.0,5% À VISTA/ 6 VEZES,TELAS,REF 08-FUSCA VERMELHO.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
8,9,WALKING,ACRÍLICA SOBRE TELA,75 x 102,2024,7500.0,7125.0,5% À VISTA/ 6 VEZES,TELAS,REF 09 - WALKING.png,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...
9,10,SUCATA,ACRÍLICA SOBRE TELA,50 X 70,2005,0.0,0.0,5% À VISTA/ 6 VEZES,TELAS,REF 10-SUCATA.jpg,acervo pessoal,6,NaN


In [336]:
with open(HTML, 'r', encoding='utf-8') as file:
    html_template = file.read()
print("Template HTML carregado com sucesso!")

Template HTML carregado com sucesso!


In [340]:
for i, row in df.iterrows():
    idx_val = int(row.get('item', i))
    pasta = f"item_{idx_val}"
    caminho_pasta = os.path.join(DIRETORIO_SAIDA, pasta)
    os.makedirs(caminho_pasta, exist_ok=True)
    link_final_da_obra = f"{URL_BASE_QR}/links_obras/{pasta}/index.html"
    df.at[i, 'qr link'] = link_final_da_obra

    titulo_encoded = urllib.parse.quote(str(row.get('telas', f'obra_{idx_val}')))
    
    tecnica_ano = f"{row.get('técnica', '')}, {row.get('ano', '')}"
    
    # Verificar se a obra está disponível para venda
    valor_credito = row.get('valor (r$)', 0)
    valor_pix_num = row.get('valor_pix', 0)
    
    # Gerar seção de pagamento ou acervo pessoal
    if valor_credito > 0 and valor_pix_num > 0:
        # Obra disponível - formatar valores e mostrar botões
        valor_credito_formatado = f"{valor_credito:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
        valor_pix_formatado = f"{valor_pix_num:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
        
        payment_section = f'''<div class="divider"></div>
        
        <!-- 💰 PAGAMENTO MERCADO PAGO -->
        <div class="payment-section">
            <p class="payment-info">{row.get('parcelamento', '').title()}</p>
            
            <div class="payment-buttons">
                <a href="{row.get('link_mp', '#')}" target="_blank" class="payment-button-credit">
                    <span class="payment-label">CARTÃO DE CRÉDITO</span>
                    <span class="payment-value">R$ {valor_credito_formatado}</span>
                    <span class="payment-installments">Parcelado em até {row.get('op_parcelamento', '6')}x</span>
                </a>
                
                <a href="{row.get('link_mp_pix', '#')}" target="_blank" class="payment-button-pix">
                    <span class="payment-label">PIX</span>
                    <span class="payment-value">R$ {valor_pix_formatado}</span>
                    <span class="payment-discount">5% de desconto à vista</span>
                </a>
            </div>
        </div>
        '''
    else:
        # Acervo pessoal - sem botões de pagamento
        payment_section = '''<div class="divider"></div>
        
        <!-- 🎨 ACERVO PESSOAL -->
        <div class="acervo-section">
            <p class="acervo-label">ACERVO PESSOAL</p>
        </div>
        '''

    # Template HTML base (sem a seção de pagamento)
    conteudo_html = f'''<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>{row.get('telas', '')} - Paulo Canuto</title>
    <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@400;700&family=Playfair+Display:wght@400;700&family=Oswald:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="../../assets/css/style.css">
</head>
<body>
    <div class="container">
        <!-- 📸 IMAGEM PRINCIPAL -->
        <div class="artwork-image-container">
            <img src="../../assets/img/{row.get('fotos', '')}" alt="{row.get('telas', '')}" class="artwork-image" />
        </div>
        
        <!-- 📝 INFORMAÇÕES DA OBRA -->
        <h1>PAULO CANUTO</h1>
        <h2 class="artwork-title">{row.get('telas', '')}</h2>
        <div class="divider"></div>
        
        <div class="details-section">
            <p><strong>{tecnica_ano}</strong></p>
            <p><strong>{row.get('dimensão', '')}</strong></p>
        </div>
        
        {payment_section}
        
        <!-- 🔗 CALL-TO-ACTION -->
        <div class="cta-section">
            <a href="https://wa.me/5571999837602?text=Tenho interesse na obra: {titulo_encoded}" target="_blank" class="whatsapp-button">
                Fale com o artista
            </a>
            <a href="https://datacanuto.github.io/galeria-digital-artes/catalog_mobile/index.html" class="catalog-button">
                CATÁLOGO COMPLETO
            </a>
        </div>
    </div>
</body>
</html>'''

    caminho_html = os.path.join(caminho_pasta, "index.html")
    with open(caminho_html, "w", encoding="utf-8") as arquivo_html:
        arquivo_html.write(conteudo_html)

    # Gerar QR Code
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(link_final_da_obra)
    qr.make(fit=True)

    imagem_qr = qr.make_image(fill_color="black", back_color="white")
    
    # Salvar QR code no diretório da obra (mantém compatibilidade)
    caminho_qr = os.path.join(caminho_pasta, "qrcode.png")
    imagem_qr.save(caminho_qr)
    
    # Exportar QR code para pasta separada com nome simplificado
    qr_export_nome = f"qr_{idx_val}.png"
    qr_export_path = os.path.join(QR_EXPORT_DIR, qr_export_nome)
    imagem_qr.save(qr_export_path)
    
    df.at[i, 'qr_export_file'] = qr_export_nome

print("Páginas HTML e QR Codes gerados com sucesso!")
print(f"QR Codes exportados para: {QR_EXPORT_DIR}")
print(f"\nResumo:")
print(f"  - Obras disponíveis: {len(df[df['valor (r$)'] > 0])}")
print(f"  - Acervo pessoal: {len(df[df['valor (r$)'] == 0])}")

Páginas HTML e QR Codes gerados com sucesso!
QR Codes exportados para: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\qr_codes_export

Resumo:
  - Obras disponíveis: 60
  - Acervo pessoal: 2


In [338]:
# Salvar CSV com links de pagamento no repositório
csv_output = os.path.join(DIRETORIO_BASE, "obras_com_links.csv")
df.to_csv(csv_output, index=False, encoding="utf-8")
print(f"CSV salvo em: {csv_output}")
df.head()

CSV salvo em: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\obras_com_links.csv


,item,telas,técnica,dimensão,ano,valor (r$),valor_pix,parcelamento,categoria,fotos,status,op_parcelamento,link_mp,qr link,qr_export_file
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,6000.0,5700.0,5% À VISTA/ 6 VEZES,TELAS,REF 01...PORTAS E JANELAS 1.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_1.png
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2024,8400.0,7980.0,5% À VISTA/ 6 VEZES,TELAS,REF 02...PORTAS E JANELAS 2.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_2.png
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2024,3860.0,3667.0,5% À VISTA/ 6 VEZES,TELAS,REF 03...PORTAS E JANELAS 3.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_3.png
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,3500.0,3325.0,5% À VISTA/ 6 VEZES,TELAS,REF 04 ...PORTAS E JANELAS 4.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_4.png
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2024,7000.0,6650.0,5% À VISTA/ 6 VEZES,TELAS,REF 05 - PORTAS E JANELAS 5.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_5.png


In [321]:
# Gerar QR Code para o catálogo completo
catalogo_link = "https://datacanuto.github.io/galeria-digital-artes/catalog_mobile/index.html"

qr_catalogo = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr_catalogo.add_data(catalogo_link)
qr_catalogo.make(fit=True)

imagem_qr_catalogo = qr_catalogo.make_image(fill_color="black", back_color="white")

# Salvar QR code do catálogo
qr_catalogo_path = os.path.join(QR_EXPORT_DIR, "qr_catalogo_completo.png")
imagem_qr_catalogo.save(qr_catalogo_path)

print(f"QR Code do catálogo gerado com sucesso!")
print(f"Salvo em: {qr_catalogo_path}")

QR Code do catálogo gerado com sucesso!
Salvo em: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\qr_codes_export\qr_catalogo_completo.png
